In [22]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data_bin = pd.read_csv('./csv/binary_classification_data.csv')

In [5]:
# Check for missing values
missing_values = data_bin.isnull().sum()

# Display the number of missing values per column
missing_values

 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
target_class                                     0
dtype: int64

In [6]:
# Display basic statistics of the dataset
data_bin.describe()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
count,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,111.079968,46.549532,0.477857,1.770279,12.614400,26.326515,8.303556,104.857709,0.091574
std,25.652935,6.843189,1.064040,6.167913,29.472897,19.470572,4.506092,106.514540,0.288432
min,5.812500,24.772042,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.929688,42.376018,0.027098,-0.188572,1.923077,14.437332,5.781506,34.960504,0.000000
50%,115.078125,46.947479,0.223240,0.198710,2.801839,18.461316,8.433515,83.064556,0.000000
75%,127.085938,51.023202,0.473325,0.927783,5.464256,28.428104,10.702959,139.309330,0.000000
max,192.617188,98.778911,8.069522,68.101622,223.392141,110.642211,34.539844,1191.000837,1.000000


StandardScaler를 사용하여 각 특성을 평균이 0이고 표준 편차가 1이 되도록 변환.   
주의할 점은, 스케일러는 학습 데이터에만 적합해야 하며, 이 스케일러를 사용하여 테스트 데이터도 변환해야 한다는 것입니다.   
이렇게 하는 이유는 테스트 데이터는 학습 과정에서 보이지 않아야 하며, 실제 모델이 처음 보는 데이터에 대한 성능을 정확하게 측정하려면 이와 같은 절차가 필요하기 때문입니다

In [7]:
# Define the features and the target
X = data_bin.drop("target_class", axis=1)
y = data_bin["target_class"]

# Initialize a new StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features and transform
X_scaled = scaler.fit_transform(X)

# Convert the scaled features into a DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Display the first few rows of the scaled features
X_scaled.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,1.149317,1.334832,-0.669570,-0.400459,-0.319440,-0.370625,-0.072798,-0.287438
1,-0.334168,1.802265,-0.011785,-0.370535,-0.371102,-0.588924,0.504427,0.211581
2,-0.314372,-1.053322,-0.145233,-0.116593,-0.322107,-0.235328,-0.125996,-0.391373
3,1.000694,1.553254,-0.513409,-0.390178,-0.304404,-0.275666,-0.312265,-0.481300
4,-0.871402,-0.858879,0.115609,-0.104866,-0.388010,-0.763111,1.324026,1.386794


각 특성이 평균 0과 표준 편차 1을 가지도록 스케일링되었습니다. 이 데이터를 이용해서 이진 분류 모델을 학습시킬 준비가 되었습니다.

다음 단계는 이 데이터를 학습 데이터셋과 테스트 데이터셋으로 분할하는 것입니다. 이를 위해서는 일반적으로 sklearn의 **`train_test_split`** 함수를 사용합니다. 분할 비율을 지정해야 하는데, 일반적으로 전체 데이터셋의 약 70~80%를 학습에 사용하고, 나머지를 테스트에 사용합니다. 이렇게 분할해보겠습니다.

In [8]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14318, 8), (3580, 8), (14318,), (3580,))

데이터가 성공적으로 분할되었습니다. 학습 데이터셋에는 총 14,318개의 관측치가 있고, 테스트 데이터셋에는 3,580개의 관측치가 있습니다. 이제 이 데이터를 사용하여 이진 분류 모델을 학습시킬 수 있습니다.

이진 분류 문제를 해결하기 위한 다양한 알고리즘이 있지만, 여기서는 로지스틱 회귀를 사용해보겠습니다. 로지스틱 회귀는 이진 분류 문제에 널리 사용되는 기법 중 하나로, 특성과 레이블 사이의 로지스틱 함수를 학습합니다.

이제 이 로지스틱 회귀 모델을 학습 데이터에 적합시켜 보겠습니다.

In [12]:
# Initialize a new logistic regression model
model = LogisticRegression(random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

로지스틱 회귀 모델이 학습 데이터에 성공적으로 적합되었습니다. 이제 이 모델을 사용하여 테스트 데이터에 대한 예측을 수행할 수 있습니다. 그 후, 모델의 성능을 평가하기 위해 여러 가지 메트릭을 사용할 수 있습니다. 일반적으로 이진 분류 문제에서는 정확도, 정밀도, 재현율, F1 점수, ROC 곡선 등을 확인합니다.

먼저, 모델을 사용하여 테스트 데이터에 대한 예측을 수행해 보겠습니다.

In [13]:
# Use the model to make predictions on the test data
y_pred = model.predict(X_test)

# Display the predictions
y_pred

array([0, 1, 0, ..., 0, 0, 1], dtype=int64)

테스트 데이터셋에 대한 예측을 수행했습니다. 이제 이 예측 결과를 사용하여 모델의 성능을 평가할 수 있습니다.

이를 위해 여러 가지 메트릭을 사용할 수 있습니다. 여기서는 정확도, 정밀도, 재현율, F1 점수를 계산해보겠습니다. 이러한 메트릭들은 모두 실제 레이블(y_test)와 예측 레이블(y_pred)을 입력으로 사용합니다.

또한, ROC 곡선 및 AUC 점수를 계산하여 모델의 성능을 시각적으로 확인할 수 있습니다. ROC 곡선은 False Positive Rate에 대한 True Positive Rate의 그래프이며, AUC(Area Under the Curve) 점수는 이 곡선 아래의 영역을 나타냅니다. AUC 점수는 0.5에서 1.0 사이의 값을 가지며, 값이 클수록 모델의 성능이 좋다는 것을 의미합니다.

이제 이러한 메트릭을 계산해보겠습니다.

In [15]:
# Calculate the metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # Compute ROC AUC from prediction scores

accuracy, precision, recall, f1, roc_auc

(0.9787709497206704,
 0.9359430604982206,
 0.8193146417445483,
 0.8737541528239202,
 0.9744632405445165)

모델의 성능 메트릭을 계산한 결과는 다음과 같습니다:

- 정확도 (Accuracy): 0.9788
- 정밀도 (Precision): 0.9359
- 재현율 (Recall): 0.8193
- F1 점수: 0.8738
- ROC AUC: 0.9745

이 메트릭들은 모델의 성능을 다양한 관점에서 평가합니다:

- **정확도**: 모든 예측 중 올바른 예측의 비율입니다. 이 경우, 약 97.88%의 예측이 정확했습니다.
- **정밀도**: Positive 클래스로 예측한 샘플 중 실제로 Positive 클래스인 샘플의 비율입니다. 이 경우, 약 93.59%의 Positive 클래스 예측이 정확했습니다.
- **재현율**: 실제 Positive 클래스인 샘플 중 Positive 클래스로 예측된 샘플의 비율입니다. 이 경우, 실제 Positive 클래스인 샘플 중 약 81.93%를 올바르게 예측했습니다.
- **F1 점수**: 정밀도와 재현율의 조화 평균입니다. 이 값은 모델의 정밀도와 재현율을 동시에 고려하므로, 불균형한 데이터셋에서 유용한 메트릭이 될 수 있습니다. 이 경우, F1 점수는 약 0.8738입니다.
- **ROC AUC**: Receiver Operating Characteristic Area Under the Curve의 약자로, 이진 분류기의 성능을 평가하는 데 사용되는 메트릭입니다. 이 값이 클수록 모델의 성능이 좋다는 것을 의미합니다. 이 경우, ROC AUC는 약 0.9745입니다.

### 하이퍼파라미터 튜닝

 하이퍼파라미터 튜닝은 모델 성능을 향상시키는 데 매우 중요한 단계입니다. 여기서는 Grid Search 방법을 사용하여 로지스틱 회귀 모델의 하이퍼파라미터를 튜닝해보겠습니다. Grid Search는 모든 가능한 하이퍼파라미터 조합을 시도하고, 가장 좋은 성능을 내는 하이퍼파라미터 조합을 찾습니다.

로지스틱 회귀의 주요 하이퍼파라미터는 다음과 같습니다:

- **C**: 이 값은 규제의 강도를 제어합니다. 값이 작을수록 모델은 더 많은 규제를 받게 되고, 값이 클수록 모델은 덜 규제를 받게 됩니다.
- **penalty**: 이 값은 사용할 규제의 종류를 결정합니다. 'l1', 'l2', 'elasticnet', 'none' 중에서 선택할 수 있습니다.

이제 이 두 하이퍼파라미터에 대해 Grid Search를 수행해 보겠습니다. 이를 위해, scikit-learn의 **`GridSearchCV`** 함수를 사용할 것입니다. 이 함수는 주어진 모델에 대해 Grid Search를 수행하고, 가장 좋은 성능을 내는 하이퍼파라미터 조합을 찾습니다. 또한, 이 함수는 교차 검증을 사용하여 모델의 성능을 평가하므로, 모델이 학습 데이터에 과적합되지 않았는지 확인할 수 있습니다.

먼저, Grid Search를 설정해보겠습니다.

In [16]:
# Define the parameter grid
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2"]
}

# Initialize a new GridSearchCV instance
grid_search = GridSearchCV(LogisticRegression(random_state=42, solver="liblinear"), param_grid, cv=5)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(random_state=42, solver='liblinear'),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2']})

Grid Search가 성공적으로 완료되었습니다. 이제 가장 좋은 성능을 낸 하이퍼파라미터 조합을 확인하고, 이를 사용하여 모델의 성능을 평가해 보겠습니다.

In [17]:
# Get the best parameters
best_params = grid_search.best_params_

# Display the best parameters
best_params

{'C': 10, 'penalty': 'l1'}

Grid Search 결과, C가 10이고 penalty가 'l1'일 때 가장 좋은 성능을 보였습니다.

이제 이 하이퍼파라미터를 사용하여 모델의 성능을 다시 평가해 보겠습니다. 이를 위해 우선 가장 좋은 성능을 낸 모델을 가져오고, 이 모델을 사용하여 테스트 데이터에 대한 예측을 수행한 후, 이전과 같은 메트릭을 사용하여 성능을 평가하겠습니다.

In [18]:
# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Use the best model to make predictions on the test data
y_pred_best = best_model.predict(X_test)

# Calculate the metrics
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])  # Compute ROC AUC from prediction scores

accuracy_best, precision_best, recall_best, f1_best, roc_auc_best

(0.979050279329609,
 0.9361702127659575,
 0.822429906542056,
 0.8756218905472636,
 0.974336106387392)

하이퍼파라미터 튜닝 후의 모델 성능 메트릭은 다음과 같습니다:

- 정확도 (Accuracy): 0.9791
- 정밀도 (Precision): 0.9362
- 재현율 (Recall): 0.8224
- F1 점수: 0.8756
- ROC AUC: 0.9743

하이퍼파라미터 튜닝 전과 비교했을 때, 모든 메트릭이 약간 향상된 것을 볼 수 있습니다. 이는 Grid Search를 통한 하이퍼파라미터 튜닝이 모델 성능 향상에 도움이 될 수 있음을 보여줍니다.

### 모델개선방안
1. **다른 모델 사용하기**: 로지스틱 회귀 외에도 여러 가지 분류 알고리즘이 있습니다. 결정 트리, 랜덤 포레스트, 그래디언트 부스팅, 서포트 벡터 머신, 신경망 등을 시도해 볼 수 있습니다. 이들 각각은 다른 유형의 데이터 패턴을 캡처하는 데 장점이 있으므로, 하나의 모델이 작동하지 않는다면 다른 모델을 시도해 보는 것이 좋습니다.
2. **하이퍼파라미터 더 튜닝하기**: 이미 Grid Search를 사용하여 하이퍼파라미터를 튜닝했지만, 더 넓은 범위의 하이퍼파라미터를 시도해 볼 수 있습니다. 또는 더 세밀한 그리드를 사용하여 특정 영역의 하이퍼파라미터를 더 자세히 탐색할 수 있습니다.
3. **데이터 전처리 개선하기**: 더 좋은 입력 특성을 만들어내는 특성 엔지니어링을 수행하거나, 이상치를 제거하거나, 데이터를 더 잘 정규화하거나 스케일링하는 등의 방법으로 데이터 전처리를 개선할 수 있습니다.
4. **데이터를 더 모으기**: 모델이 더 많은 패턴을 학습할 수 있도록 데이터를 더 많이 모으는 것이 도움이 될 수 있습니다. 이는 특히 복잡한 모델에서 유용할 수 있습니다.
5. **클래스 불균형 처리하기**: 만약 데이터셋에 클래스 불균형이 있으면, 소수 클래스에 대한 모델의 성능이 저하될 수 있습니다. 이를 해결하기 위해 오버샘플링, 언더샘플링, SMOTE 등의 방법을 사용할 수 있습니다.

특성 선택에는 여러 가지 방법이 있지만, 여기서는 두 가지 기본적인 방법을 사용해보겠습니다:

1. **상관 관계 분석**: 특성과 목표 변수 사이의 상관 관계를 확인하고, 목표 변수와 높은 상관 관계를 가지는 특성만 선택합니다.
2. **재귀적 특성 제거 (Recursive Feature Elimination, RFE)**: 이 방법은 모델을 학습시키고, 가장 중요하지 않은 특성을 하나씩 제거하는 과정을 반복합니다. 이 과정은 목표로 하는 특성의 개수가 남을 때까지 계속됩니다.

이제 특성 선택을 위해 상관 관계 분석을 수행해 보겠습니다. 이를 위해 우선 각 특성과 목표 변수 사이의 상관 관계를 계산하겠습니다.

In [20]:

# Calculate the correlation between each feature and the target
correlations = data_bin.corr()["target_class"].sort_values()

# Display the correlations
correlations

 Mean of the integrated profile                 -0.673181
 Excess kurtosis of the DM-SNR curve            -0.390816
 Standard deviation of the integrated profile   -0.363708
 Skewness of the DM-SNR curve                   -0.259117
 Mean of the DM-SNR curve                        0.400876
 Standard deviation of the DM-SNR curve          0.491535
 Skewness of the integrated profile              0.709528
 Excess kurtosis of the integrated profile       0.791591
target_class                                     1.000000
Name: target_class, dtype: float64

각 특성과 목표 클래스 (펄서인지 아닌지) 사이의 상관 관계를 계산하였습니다. 상관 계수는 -1에서 1 사이의 값을 가지며, 값이 -1에 가까울수록 강한 음의 상관 관계, 값이 1에 가까울수록 강한 양의 상관 관계를 나타냅니다. 0은 두 변수 사이에 상관 관계가 없음을 나타냅니다.

이 결과를 보면, 'Excess kurtosis of the integrated profile'와 'Skewness of the integrated profile' 특성이 목표 클래스와 가장 강한 양의 상관 관계를 가지고 있음을 알 수 있습니다. 반면에 'Mean of the integrated profile' 특성은 목표 클래스와 강한 음의 상관 관계를 가지고 있습니다.

이제 이 정보를 바탕으로 특성 선택을 수행할 수 있습니다. 예를 들어, 상관 계수가 특정 임계값 이상인 특성만 선택할 수 있습니다. 또는, 상관 계수가 가장 높은 n개의 특성만 선택할 수 있습니다.

다음 단계로, 재귀적 특성 제거 (RFE)를 사용하여 특성 선택을 수행해보겠습니다. 이를 위해 sklearn의 **`RFECV`** 함수를 사용하겠습니다. 이 함수는 교차 검증을 사용하여 가장 좋은 특성의 개수를 찾습니다. 이제 이 함수를 사용하여 RFE를 수행해 보겠습니다.

In [23]:
# Separate the features and the target
X = data_bin.drop("target_class", axis=1)
y = data_bin["target_class"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize a new StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data
scaler.fit(X_train)

# Scale the training data
X_train = scaler.transform(X_train)

# Scale the test data
X_test = scaler.transform(X_test)

# Initialize an RFECV instance
selector = RFECV(LogisticRegression(random_state=42), step=1, cv=5)

# Fit the selector to the data
selector.fit(X_train, y_train)

# Get the mask of selected features
selected_features_mask = selector.support_

# Apply the mask to get the selected features
selected_features = X.columns[selected_features_mask]

# Display the selected features
selected_features

Index([' Mean of the integrated profile',
       ' Standard deviation of the integrated profile',
       ' Excess kurtosis of the integrated profile',
       ' Skewness of the integrated profile', ' Mean of the DM-SNR curve',
       ' Standard deviation of the DM-SNR curve',
       ' Excess kurtosis of the DM-SNR curve',
       ' Skewness of the DM-SNR curve'],
      dtype='object')

재귀적 특성 제거 (RFE)를 사용하여 특성 선택을 수행한 결과, 다음 특성들이 선택되었습니다:

- Mean of the integrated profile
- Standard deviation of the integrated profile
- Excess kurtosis of the integrated profile
- Skewness of the integrated profile
- Mean of the DM-SNR curve
- Standard deviation of the DM-SNR curve
- Excess kurtosis of the DM-SNR curve
- Skewness of the DM-SNR curve

원래 데이터셋에는 총 8개의 특성이 있었으므로, RFE가 모든 특성을 유용하다고 판단한 것입니다.

이제 이 선택된 특성들로 모델을 다시 학습시키고, 성능을 평가해 볼 수 있습니다. 이를 위해 우선 선택된 특성들로 학습 데이터와 테스트 데이터를 재구성하겠습니다. 그리고 이 데이터를 사용하여 로지스틱 회귀 모델을 다시 학습시키고, 테스트 데이터에 대한 예측을 수행한 후, 성능 메트릭을 계산하겠습니다.

In [26]:
# Select the features from the training and test sets
X_train_selected = X_train[:, selected_features_mask]
X_test_selected = X_test[:, selected_features_mask]

# Initialize a new LogisticRegression instance
model_selected = LogisticRegression(random_state=42)

# Fit the model to the training data
model_selected.fit(X_train_selected, y_train)

# Use the model to predict the outcomes on the test set
y_pred_selected = model_selected.predict(X_test_selected)


In [28]:
# Calculate the metrics
accuracy_selected = accuracy_score(y_test, y_pred_selected)
precision_selected = precision_score(y_test, y_pred_selected)
recall_selected = recall_score(y_test, y_pred_selected)
f1_selected = f1_score(y_test, y_pred_selected)
roc_auc_selected = roc_auc_score(y_test, model_selected.predict_proba(X_test_selected)[:, 1])  # Compute ROC AUC from prediction scores

accuracy_selected, precision_selected, recall_selected, f1_selected, roc_auc_selected

(0.979608938547486,
 0.9442508710801394,
 0.8262195121951219,
 0.88130081300813,
 0.9710009600096001)